In [1]:
import json
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm.notebook import trange, tqdm
from datetime import datetime
import csv

In [2]:
def load_llama_model():
    """Load LLaMA model and tokenizer"""
    model_name = "/home/g4/Llama-3.2-3B-Instruct-Finetuned-combined/"
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    if model.config.pad_token_id is None:
        model.config.pad_token_id = model.config.eos_token_id

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    model = model.to(device)

    return model, tokenizer, device

model, tokenizer, device = load_llama_model()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using device: cuda


In [3]:
distortions = {
    "fortune_telling": "Fortune Telling/Catastrophizing",
    "what_if_statements": "What If Statements",
    "labeling": "Labeling/Global Labeling",
    "unfair_comparisons": "Unfair Comparisons",
    "mind_reading_conclusion": "Mind Reading",
    "should_statements_conclusion": "Should Statements",
    "overgeneralization": "Overgeneralization",
    "all_or_nothing_thinking": "All or Nothing Thinking",
    "blaming_conclusion": "Blaming",
    "emotional_reasoning": "Emotional Reasoning",
    "mental_filter": "Mental Filter",
    "discounting_the_positive": "Discounting the Positive",
    "magnification_minimization": "Magnification/Minimization",
    "personalization_conclusion": "Personalization",
    "jumping_to_conclusions": "Jumping to Conclusions",
    "no_distortion": "No Distortion"
}

name_to_node = {
    "Fortune Telling": "fortune_telling",
    "What if?": "what_if_statements",
    "Labeling/Global Labeling": "labeling",
    "Unfair Comparisons": "unfair_comparisons",
    "Mind Reading": "mind_reading_conclusion",
    "Should Statements": "should_statements_conclusion",
    "Overgeneralization": "overgeneralization",
    "All or Nothing Thinking": "all_or_nothing_thinking",
    "Blaming": "blaming_conclusion",
    "Emotional Reasoning": "emotional_reasoning",
    "Mental Filter": "mental_filter",
    "Discounting the Positive": "discounting_the_positive",
    "Magnification/Minimization": "magnification_minimization",
    "Personalization": "personalization_conclusion",
    "Jumping to Conclusions": "jumping_to_conclusions",
    "No Distortion": "no_distortion"
}

# Define the decision tree structure
decision_tree = {
    "root": {
        "question": "Is the thought pattern about future events or predictions?",
        "yes": "future",
        "no": "evidence_check"
    },
    "evidence_check": {
        "question": "Is the thought pattern about making conclusions or assumptions?",
        "yes": "evidence",
        "no": "evaluation_check"
    },
    "evaluation_check": {
        "question": "Is the thought pattern about evaluating or judging self/others/situations?",
        "yes": "evaluation",
        "no": "attention_check"
    },
    "attention_check": {
        "question": "Is the thought pattern about focusing attention on specific aspects?",
        "yes": "attention",
        "no": "responsibility_check"
    },
    "responsibility_check": {
        "question": "Is the thought pattern about assigning responsibility or causation?",
        "yes": "responsibility",
        "no": "no_distortion"
    },
    "future": {
        "question": "Are the predictions catastrophic or unbearable?",
        "yes": "fortune_telling",
        "no": "what_if"
    },
    "what_if": {
        "question": "Are there repeated 'what if' questions?",
        "yes": "what_if_statements",
        "no": "no_distortion"
    },
    "evidence": {
        "question": "Are conclusions drawn without sufficient evidence?",
        "yes": "jumping_to_conclusions",
        "no": "evidence_type"
    },
    "evidence_type": {
        "question": "Are emotions used as the primary evidence?",
        "yes": "emotional_reasoning",
        "no": "assumed_thoughts"
    },
    "assumed_thoughts": {
        "question": "Are others' thoughts/intentions assumed without evidence?",
        "yes": "mind_reading",
        "no": "no_distortion"
    },
    "evaluation": {
        "question": "Does the evaluation involve extreme categories?",
        "yes": "extremes",
        "no": "standards_check"
    },
    "extremes": {
        "question": "Does it use words like 'always', 'never', 'every time'?",
        "yes": "overgeneralization",
        "no": "two_options"
    },
    "two_options": {
        "question": "Is everything sorted into only two categories?",
        "yes": "all_or_nothing",
        "no": "no_distortion"
    },
    "standards_check": {
        "question": "Is it about how things 'should' be?",
        "yes": "should_statements",
        "no": "comparison_check"
    },
    "comparison_check": {
        "question": "Is it about comparing to others?",
        "yes": "unfair_comparisons",
        "no": "labels_check"
    },
    "labels_check": {
        "question": "Is it about applying fixed, global labels?",
        "yes": "labeling",
        "no": "no_distortion"
    },
    "attention": {
        "question": "Is there selective focus on specific aspects?",
        "yes": "mental_filter",
        "no": "positive_negative_check"
    },
    "positive_negative_check": {
        "question": "Is it about weighing positives versus negatives?",
        "yes": "weighing_check",
        "no": "no_distortion"
    },
    "weighing_check": {
        "question": "Is it exclusively focusing on negatives?",
        "yes": "magnification_minimization",
        "no": "positive_dismiss"
    },
    "positive_dismiss": {
        "question": "Are positive experiences being dismissed?",
        "yes": "discounting_the_positive",
        "no": "no_distortion"
    },
    "responsibility": {
        "question": "Is external behavior seen as personally directed?",
        "yes": "personalization",
        "no": "blame_check"
    },
    "blame_check": {
        "question": "Is there complete attribution of responsibility to self or others?",
        "yes": "blaming",
        "no": "no_distortion"
    }
}

distortion_definitions = """All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
Fortune telling: I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
Mental Filter: I pay attention to one or a few details and fail to see the whole picture.
Mind reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
Overgeneralization: I take isolated negative cases and generalize them, transforming them in a never-ending pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
Personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
Should statements (also “musts”, “oughts”, “have tos”): I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
Blaming (others or oneself): I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What if?: I keep asking myself questions such as “what if something happens?”
Discounting the positive: I disqualify positive experiences or events insisting that they do not count.
Magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
Jumping to conclusions: I draw conclusions (negative or positive) from little or no confirmatory evidence.
Unfair comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""
system_message = """You are a helpful assistant that can only respond with "Yes" or "No". Follow these rules:
1. You must answer with exactly "Yes" or "No" (case-sensitive)
2. Do not add any other words or punctuation
3. If you are not completely certain, answer "No"
4. These are the only two valid responses: "Yes" or "No\""""

def is_leaf_node(node_id: str) -> bool:
    """Check if a node is a leaf node (final distortion)."""
    return isinstance(decision_tree[node_id], str)

def get_distortion_description(distortion: str) -> str:
    """Get the description of a specific distortion."""
    return distortions.get(distortion, "Unknown distortion")

In [4]:
def generate_response(text: str, count) -> str:
    messages = [
    {
        "role": "system",
        "content": system_message,
    },
    {"role": "user", "content": text},
 ]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, return_dict=True, add_generation_prompt=True, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    outputs = model.generate(
        **inputs,
        max_new_tokens=count,
        temperature=0.1,
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True,
    )
    return tokenizer.decode(outputs[0][len(inputs["input_ids"][0]):])

In [ ]:
translate = {
    'All-or-nothing thinking' : 'all_or_nothing_thinking',
    'Emotional Reasoning': 'emotional_reasoning',
    'Fortune-telling': 'fortune_telling',
    'Labeling': 'labeling',
    'Magnification': 'magnification_minimization',
    'Mental filter': 'mental_filter',
    'Mind Reading': 'mind_reading_conclusion',
    'Overgeneralization': 'overgeneralization',
    "No Distortion": "",
    "Personalization": 'personalization_conclusion',
    "Should statements": "should_statements_conclusion",
    "Emotional Reasoning": "emotional_reasoning",
    "": ''
}

def make_question(text: str, question: str):
    return f'Given the following text:\n\n{text}\n\n Answer the question: {question}\n\n'

total = 0
err = 0
with open('/home/g4/Downloads/Annotated_data - Annotated_data.csv', 'r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)
    for row in tqdm(csv_reader):
        text=row[1]
        o1=translate[row[3]]
        o2=translate[row[4]]
        # print(text)
        node = 'root'
        while node in decision_tree:
            current = decision_tree[node]
            response = generate_response(make_question(text, current['question']), 1)
            # print(current['question'], response)
            if "yes" in response.lower():
                node = current['yes']
            else:
                node = current['no']

        if node in translate.values():
            total = total + 1
            if node != o1 and node != o2:
                print(node, o1, o2)
                err = err + 1
            print(float(err)/total)

    print(err, total)


0it [00:00, ?it/s]

ZeroDivisionError: float division by zero